In [ ]:
import math
import os
import time
import xml.etree.ElementTree as ElementTree
from html import unescape
from subprocess import call

from pytube import YouTube

In [ ]:
video_path = os.getcwd() + '/video_files/'
audio_path = os.getcwd() + '/audio_files/'
final_path = os.getcwd() + '/final_files/'
caption_path = os.getcwd() + '/captions/'

In [ ]:
link = input("Enter a YouTube link: ")
yt_obj = YouTube(link)

In [ ]:
#Title of video
print("Title: ",yt_obj.title)
#Number of views of video
print("Number of views: ",yt_obj.views)
#Length of the video
print("Length of video: ",yt_obj.length,"seconds")
#Rating
print("Ratings: ",yt_obj.rating)

In [ ]:
yt_obj.title = "TWICE-SET ME FREE"

In [ ]:
filter_set = 'abcdefghijklmnopqrstuvwxyz'
filter_set = filter_set + filter_set.upper() + ''.join(list(map(str, range(0, 10)))) + ' -'

title = '_'.join(''.join([x for x in yt_obj.title if x in filter_set]).split())

print(title)

In [ ]:
video_files = list()
audio_files = list()

for file in yt_obj.streams.filter(adaptive=True):

    if 'video' in file.mime_type:
        if int(file.resolution[:-1]) >= 1080:
            video_files.append(file)
    else:
        audio_files.append(file)

In [ ]:
best_video = video_files[0]

for video in video_files[1:]:

    if int(video.resolution[:-1]) > int(best_video.resolution[:-1]) and int(video.fps[:-3]) > int(best_video.fps[:-3]):
        best_video = video

best_audio = audio_files[0]

for audio in audio_files[1:]:

    if int(audio.abr[:-4]) > int(best_audio.abr[:-4]):
        best_audio = audio

print(best_video, best_audio, sep='\n')

In [ ]:
def float_to_srt_time_format(d: float) -> str:

    fraction, whole = math.modf(d)
    time_fmt = time.strftime("%H:%M:%S,", time.gmtime(whole))
    ms = f"{fraction:.3f}".replace("0.", "")
    return time_fmt + ms

def xml_caption_to_srt(xml_captions: str):

    segments = []
    root = ElementTree.fromstring(xml_captions)
    segment_number = 0

    for child in list(root.iter("body"))[0]:

        if child.tag == 'p':

            caption = child.text

            for s in list(child):
                if s.tag == 's':
                    caption += ' ' + s.text

            caption = unescape(caption.replace('\n', '').replace('  ', ' '),)

            try:
                duration = float(child.attrib['d'])/1000.0
            except KeyError:
                duration = 0.0

            start = float(child.attrib['t'])/1000.0
            end = start + duration
            segment_number += 1

            line = '{seg_num}\n{start} --> {end}\n{cap}\n'.format(
                seg_num = segment_number,
                start = float_to_srt_time_format(start),
                end = float_to_srt_time_format(end),
                cap = caption,
            )

            segments.append(line.strip())

    return '\n\n'.join(segments).strip()

In [ ]:
caption_track = None

for caption in yt_obj.caption_tracks:

    if 'en' in caption.code and 'auto' not in caption.name:
        caption_track = yt_obj.captions[caption.code]

        # print(caption_track.xml_captions)

        with open(caption_path+title+'.srt', 'w') as fp:
            fp.write(xml_caption_to_srt(caption_track.xml_captions))

In [ ]:
message = ''

def progress_callback(stream, chunk, bytes_remaining):

    try:
        size = video.filesize
        message = 'Video File Downloaded'
    except Exception as e:
        size = audio.filesize
        message = 'Audio File Downloaded'

    progress = int(((size*10 - bytes_remaining)) / 1000000)
    print(f'Total: {int(size / 100000)}MB | {progress}MB Completed', end='\r')

def complete_callback(stream, file_handle):
    print(message)

yt_obj.register_on_progress_callback(progress_callback)
yt_obj.register_on_complete_callback(complete_callback)

best_video.download(output_path=video_path, filename=title+'.'+best_video.mime_type[6:])
print()
best_audio.download(output_path=audio_path, filename=title+'.'+best_audio.mime_type[6:])

In [ ]:
video_stream = video_path+title+'.'+best_video.mime_type[6:]
audio_stream = audio_path+title+'.'+best_audio.mime_type[6:]

path = final_path+title+'.'+best_video.mime_type[6:]

print(video_stream, audio_stream, path)

caption_stream = None

if title+'.srt' in os.listdir(caption_path):
    caption_stream = caption_path+title+'.srt'

    call(f'ffmpeg -y -i {video_stream} -i {audio_stream} -i {caption_stream} -acodec copy -vcodec copy -disposition:s:0 default {path}')

    os.remove(caption_stream)

    # ffmpeg.output(audio_stream, video_stream, caption_stream, filename=path, acodec='copy', vcodec='copy').run(overwrite_output=True)
else:

    call(f'ffmpeg -y -i {video_stream} -i {audio_stream} -acodec copy -vcodec copy {path}')

    # ffmpeg.output(audio_stream, video_stream, filename=path, acodec='copy', vcodec='copy').run(overwrite_output=True)

os.remove(video_stream)
os.remove(audio_stream)